In [1]:
import pandas as pd
import spacy

In [2]:
nlp = spacy.load('pt_core_news_sm')

In [3]:
dados_treino = pd.read_csv("data/treino.csv")
dados_treino.head()

,title,text,date,category,subcategory,link
0,"Após polêmica, Marine Le Pen diz que abomina n...",A candidata da direita nacionalista à Presidên...,2017-04-28,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/04/187...
1,"Macron e Le Pen vão ao 2º turno na França, em ...",O centrista independente Emmanuel Macron e a d...,2017-04-23,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/04/187...
2,"Apesar de larga vitória nas legislativas, Macr...",As eleições legislativas deste domingo (19) na...,2017-06-19,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/06/189...
3,"Governo antecipa balanço, e Alckmin anuncia qu...",O número de ocorrências de homicídios dolosos ...,2015-07-24,cotidiano,NaN,http://www1.folha.uol.com.br/cotidiano/2015/07...
4,"Após queda em maio, a atividade econômica sobe...","A economia cresceu 0,25% no segundo trimestre,...",2017-08-17,mercado,NaN,http://www1.folha.uol.com.br/mercado/2017/08/1...


In [4]:
texto = "Rio de !!!janeiro$$$$$ é uma cidade maravilhosa"
doc = nlp(texto)

In [5]:
textos_para_tratamento = (titulos.lower() for titulos in dados_treino["title"])

In [6]:
def trata_textos(doc):
    tokens_validos = []
    for token in doc:
        e_valido = not token.is_stop and token.is_alpha
        if e_valido:
            tokens_validos.append(token.text)
        
    if len(tokens_validos) > 2:
        return " ".join(tokens_validos)

In [7]:
trata_textos(doc)

'Rio cidade maravilhosa'

In [8]:
from time import time

# pegar o início da execução (para calcular o tempo levado para o processamento)
t0 = time()

# processar em paralelo e enviar em lotes
textos_tratados = [trata_textos(doc) for doc in nlp.pipe(textos_para_tratamento,
                                                       batch_size = 1000,
                                                       n_process = -1)]

tf = time() - t0

print(tf/60)

3.688579992453257


In [10]:
titulos_tratados = pd.DataFrame({"titulo":textos_tratados})

titulos_tratados.head()

,titulo
0,polêmica marine le pen abomina negacionistas h...
1,macron le pen turno frança revés siglas tradic...
2,apesar larga vitória legislativas macron terá ...
3,governo antecipa balanço alckmin anuncia queda...
4,queda maio atividade econômica sobe junho bc


In [11]:
from gensim.models import Word2Vec

w2v_modelo = Word2Vec(sg = 0,
                     window = 2,
                     #size = 300,
                     min_count = 5,
                     alpha = 0.03,
                     min_alpha = 0.007)

In [12]:
w2v_modelo

In [13]:
print(len(titulos_tratados))

titulos_tratados = titulos_tratados.dropna().drop_duplicates()

print(len(titulos_tratados))

90000
84466


In [14]:
lista_lista_tokens = [titulo.split(" ") for titulo in titulos_tratados.titulo]

In [15]:
import logging

logging.basicConfig(format="%(asctime)s : - %(message)s", level = logging.INFO)

w2v_modelo = Word2Vec(sg = 0,
                     window = 2,
                     #size = 300,
                     min_count = 5,
                     alpha = 0.03,
                     min_alpha = 0.007)

w2v_modelo.build_vocab(lista_lista_tokens, progress_per=5000)

2021-09-15 18:16:27,319 : - Word2Vec lifecycle event {'params': 'Word2Vec(vocab=0, vector_size=100, alpha=0.03)', 'datetime': '2021-09-15T18:16:27.319522', 'gensim': '4.1.1', 'python': '3.7.12 (default, Sep  8 2021, 01:20:16) \n[GCC 10.2.1 20210110]', 'platform': 'Linux-5.4.72-microsoft-standard-WSL2-x86_64-with-debian-11.0', 'event': 'created'}
2021-09-15 18:16:27,323 : - collecting all words and their counts
2021-09-15 18:16:27,325 : - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-09-15 18:16:27,347 : - PROGRESS: at sentence #5000, processed 31930 words, keeping 10193 word types
2021-09-15 18:16:27,364 : - PROGRESS: at sentence #10000, processed 63848 words, keeping 14989 word types
2021-09-15 18:16:27,386 : - PROGRESS: at sentence #15000, processed 95753 words, keeping 18279 word types
2021-09-15 18:16:27,408 : - PROGRESS: at sentence #20000, processed 127689 words, keeping 21033 word types
2021-09-15 18:16:27,423 : - PROGRESS: at sentence #25000, processed 

In [16]:
w2v_modelo.train(lista_lista_tokens, 
                 total_examples=w2v_modelo.corpus_count,
                 epochs = 30)

2021-09-15 18:18:32,234 : - Word2Vec lifecycle event {'msg': 'training model with 3 workers on 12924 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=2 shrink_windows=True', 'datetime': '2021-09-15T18:18:32.234646', 'gensim': '4.1.1', 'python': '3.7.12 (default, Sep  8 2021, 01:20:16) \n[GCC 10.2.1 20210110]', 'platform': 'Linux-5.4.72-microsoft-standard-WSL2-x86_64-with-debian-11.0', 'event': 'train'}
2021-09-15 18:18:32,824 : - worker thread finished; awaiting finish of 2 more threads
2021-09-15 18:18:32,846 : - worker thread finished; awaiting finish of 1 more threads
2021-09-15 18:18:32,847 : - worker thread finished; awaiting finish of 0 more threads
2021-09-15 18:18:32,849 : - EPOCH - 1 : training on 540242 raw words (486192 effective words) took 0.6s, 822110 effective words/s
2021-09-15 18:18:33,546 : - worker thread finished; awaiting finish of 2 more threads
2021-09-15 18:18:33,577 : - worker thread finished; awaiting finish of 1 more threads
2021-09

2021-09-15 18:18:45,102 : - worker thread finished; awaiting finish of 2 more threads
2021-09-15 18:18:45,106 : - worker thread finished; awaiting finish of 1 more threads
2021-09-15 18:18:45,128 : - worker thread finished; awaiting finish of 0 more threads
2021-09-15 18:18:45,130 : - EPOCH - 21 : training on 540242 raw words (486133 effective words) took 0.6s, 776824 effective words/s
2021-09-15 18:18:45,894 : - worker thread finished; awaiting finish of 2 more threads
2021-09-15 18:18:45,906 : - worker thread finished; awaiting finish of 1 more threads
2021-09-15 18:18:45,915 : - worker thread finished; awaiting finish of 0 more threads
2021-09-15 18:18:45,917 : - EPOCH - 22 : training on 540242 raw words (486111 effective words) took 0.8s, 630767 effective words/s
2021-09-15 18:18:46,508 : - worker thread finished; awaiting finish of 2 more threads
2021-09-15 18:18:46,521 : - worker thread finished; awaiting finish of 1 more threads
2021-09-15 18:18:46,528 : - worker thread finished

(14584327, 16207260)

In [26]:
## Avaliação Qualitativa
w2v_modelo.wv.most_similar("pfizer")

[('tornozeleiras', 0.6468854546546936),
 ('varejistas', 0.6299357414245605),
 ('leniência', 0.6258296370506287),
 ('kraft', 0.6175836324691772),
 ('invepar', 0.6105926632881165),
 ('telecomunicações', 0.6027239561080933),
 ('sabmiller', 0.5957454442977905),
 ('citibank', 0.5954554080963135),
 ('negociam', 0.5772911906242371),
 ('braskem', 0.577210545539856)]

In [28]:
## Análise Qualitativa
w2v_modelo.wv.most_similar("microsoft")

[('unilever', 0.5758864879608154),
 ('amazon', 0.572556734085083),
 ('buffett', 0.5339102745056152),
 ('sony', 0.5299050211906433),
 ('viajante', 0.5268521308898926),
 ('ikea', 0.5176929235458374),
 ('sky', 0.511826753616333),
 ('lego', 0.5113440155982971),
 ('editora', 0.5045936107635498),
 ('tesla', 0.501508891582489)]

In [32]:
#Treinamento do modelo Skip-gram
# Skip-Gram possui um resultado melhor com um window maior, por isso, alterado de 2 para 5

w2v_modelo_sg = Word2Vec(sg = 1,
                     window = 5,
                     #size = 300,
                     min_count = 5,
                     alpha = 0.03,
                     min_alpha = 0.007)

w2v_modelo_sg.build_vocab(lista_lista_tokens, progress_per=5000)

w2v_modelo_sg.train(lista_lista_tokens,
                 total_examples=w2v_modelo_sg.corpus_count,
                 epochs = 30)

2021-09-15 18:32:33,160 : - Word2Vec lifecycle event {'params': 'Word2Vec(vocab=0, vector_size=100, alpha=0.03)', 'datetime': '2021-09-15T18:32:33.159991', 'gensim': '4.1.1', 'python': '3.7.12 (default, Sep  8 2021, 01:20:16) \n[GCC 10.2.1 20210110]', 'platform': 'Linux-5.4.72-microsoft-standard-WSL2-x86_64-with-debian-11.0', 'event': 'created'}
2021-09-15 18:32:33,163 : - collecting all words and their counts
2021-09-15 18:32:33,164 : - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-09-15 18:32:33,178 : - PROGRESS: at sentence #5000, processed 31930 words, keeping 10193 word types
2021-09-15 18:32:33,198 : - PROGRESS: at sentence #10000, processed 63848 words, keeping 14989 word types
2021-09-15 18:32:33,213 : - PROGRESS: at sentence #15000, processed 95753 words, keeping 18279 word types
2021-09-15 18:32:33,226 : - PROGRESS: at sentence #20000, processed 127689 words, keeping 21033 word types
2021-09-15 18:32:33,242 : - PROGRESS: at sentence #25000, processed 

2021-09-15 18:32:42,760 : - EPOCH - 8 : training on 540242 raw words (486138 effective words) took 1.2s, 404918 effective words/s
2021-09-15 18:32:43,775 : - EPOCH 9 - PROGRESS: at 94.38% examples, 456245 words/s, in_qsize 4, out_qsize 0
2021-09-15 18:32:43,814 : - worker thread finished; awaiting finish of 2 more threads
2021-09-15 18:32:43,825 : - worker thread finished; awaiting finish of 1 more threads
2021-09-15 18:32:43,836 : - worker thread finished; awaiting finish of 0 more threads
2021-09-15 18:32:43,837 : - EPOCH - 9 : training on 540242 raw words (486170 effective words) took 1.1s, 455080 effective words/s
2021-09-15 18:32:44,853 : - EPOCH 10 - PROGRESS: at 77.75% examples, 375424 words/s, in_qsize 5, out_qsize 0
2021-09-15 18:32:45,139 : - worker thread finished; awaiting finish of 2 more threads
2021-09-15 18:32:45,145 : - worker thread finished; awaiting finish of 1 more threads
2021-09-15 18:32:45,163 : - worker thread finished; awaiting finish of 0 more threads
2021-09

2021-09-15 18:33:08,793 : - worker thread finished; awaiting finish of 2 more threads
2021-09-15 18:33:08,816 : - worker thread finished; awaiting finish of 1 more threads
2021-09-15 18:33:08,836 : - worker thread finished; awaiting finish of 0 more threads
2021-09-15 18:33:08,837 : - EPOCH - 25 : training on 540242 raw words (486131 effective words) took 1.4s, 355750 effective words/s
2021-09-15 18:33:09,858 : - EPOCH 26 - PROGRESS: at 70.35% examples, 338083 words/s, in_qsize 5, out_qsize 0
2021-09-15 18:33:10,204 : - worker thread finished; awaiting finish of 2 more threads
2021-09-15 18:33:10,222 : - worker thread finished; awaiting finish of 1 more threads
2021-09-15 18:33:10,241 : - worker thread finished; awaiting finish of 0 more threads
2021-09-15 18:33:10,242 : - EPOCH - 26 : training on 540242 raw words (486131 effective words) took 1.4s, 348332 effective words/s
2021-09-15 18:33:11,266 : - EPOCH 27 - PROGRESS: at 68.53% examples, 329505 words/s, in_qsize 5, out_qsize 0
2021

(14583764, 16207260)

In [33]:
w2v_modelo_sg.wv.most_similar("google")

[('facebook', 0.680336594581604),
 ('apple', 0.6156793236732483),
 ('chips', 0.5675997138023376),
 ('reguladores', 0.5611848831176758),
 ('amazon', 0.549229621887207),
 ('android', 0.5440897941589355),
 ('volkswagen', 0.5362513661384583),
 ('waze', 0.5348283648490906),
 ('concorda', 0.5347558856010437),
 ('buffett', 0.5292649269104004)]

In [39]:
def matriz_vetores(textos):
    x = len(textos)
    y = 300
    matriz = np.zeros((x,y))
    
    for i in range(x):
        palavras_numeros = tokenizador(textos.iloc[i])
        matriz[i] = combinacao_de_vetores_por_soma(palavras_numeros)
        
    return matriz

matriz_vetores_treino = matriz_vetores(artigo_treino.title)
matriz_vetores_teste = matriz_vetores(artigo_treino.title)
print(matriz_vetores_treino)
print(matriz_vetores_teste)

NameError: name 'artigo_treino' is not defined

In [ ]:
def combinacao_vetores_por_soma()

In [35]:
w2v_modelo.wv.save_word2vec_format("modelos/modelo_cbow.txt", binary=False)
w2v_modelo_sg.wv.save_word2vec_format("modelos/modelo_skipgram.txt", binary=False)

2021-09-15 18:41:28,433 : - storing 12924x100 projection weights into modelos/modelo_cbow.txt
2021-09-15 18:41:29,769 : - storing 12924x100 projection weights into modelos/modelo_skipgram.txt


In [38]:
import string
import nltk

def tokenizador(texto):
    texto = texto.lower()
    lista_alfanumerico = []
    
    for token_valido in nltk.word_tokenize(texto):
        if token_valido in string.ponctuation: continue
        lista_alfanumerico.append(token_valido)
            
    return lista_alfanumerico

tokenizador("Texto Exemplo, 1234.")

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/PY3/english.pickle[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/local/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/local/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************
